In [3]:
# Import necessary libraries
import json
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


In [4]:

# Load the dataset
with open('dataset.json', 'r') as file:
    data = json.load(file)

# Data Preprocessing
sentences = []
labels = []
classes = set()

for intent in data['intents']:
    for pattern in intent['patterns']:
        sentences.append(pattern)
        labels.append(intent['tag'])
    classes.add(intent['tag'])

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

max_len = max([len(x) for x in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Model building
model = Sequential()
model.add(Embedding(2000, 10, input_length=max_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(classes), activation='softmax'))



# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/home/anudeep/Documents/NLP/Project/vir/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1727891409.643292    6579 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-02 23:20:09.688740: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:


# Train the model
model.fit(padded_sequences, np.array(encoded_labels), epochs=200, batch_size=8, validation_split=0.2)

# Save the model and tokenizer
model.save('chatbot_model.h5')

with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pkl', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)


Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.0972 - loss: 2.3965 - val_accuracy: 0.0000e+00 - val_loss: 2.4454
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1541 - loss: 2.3819 - val_accuracy: 0.0000e+00 - val_loss: 2.5262
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1072 - loss: 2.3629 - val_accuracy: 0.0000e+00 - val_loss: 2.7325
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1811 - loss: 2.3388 - val_accuracy: 0.0000e+00 - val_loss: 3.1176
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1028 - loss: 2.2698 - val_accuracy: 0.0000e+00 - val_loss: 3.8028
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1072 - loss: 2.2790 - val_accuracy: 0.0000e+00 - val_loss: 3.6359
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1009 - loss: 2.2202 - val_accuracy: 0.0000e+00 - val_loss: 3.8506
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1770 - loss: 2.2220 - val_

In [6]:

# Chatbot interaction function
def chatbot_response(user_input):
    seq = tokenizer.texts_to_sequences([user_input])
    padded = pad_sequences(seq, maxlen=max_len)
    pred = np.argmax(model.predict(padded), axis=-1)
    tag = label_encoder.inverse_transform(pred)[0]

    for intent in data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])



In [7]:
# Example interaction
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = chatbot_response(user_input)
    print("Bot:", response)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
Bot: To reset your router, unplug it from the power source, wait 30 seconds, then plug it back in. Allow a few minutes for the connection to reestablish.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Bot: Resetting your router restores it to its default settings, which may resolve connection problems caused by incorrect configurations.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: Check if your internet connection is stable before connecting to the VPN. VPN performance depends on your base connection speed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot: Ensure no other devices are consuming too much bandwidth. Also, check for physical obstructions between your device and the Wi-Fi router.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: Resetting your router restores it to its default settings, which may resolve connection problems caused by incorrect configurations.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Bot: Resetting your router restores it to its default settings, 